### Add scripts path to the notebook

In [1]:
import sys
import os

current_dir = os.getcwd()
print(current_dir)

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

scripts_path = os.path.join(parent_dir, 'scripts')

# Insert the path to the parent directory
sys.path.insert(0, parent_dir)

# Insert the path to the Scripts directory
sys.path.insert(0, scripts_path)

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..')))

d:\KifiyaAIM-Course\Week - 3\ACIS-Car-Insurance-Claim-Analysis\notebooks


### Import Statements

In [32]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.cluster import KMeans
from sklearn.preprocessing import Normalizer

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

### Load the data

In [3]:
PATH_TO_DATA = "../data/MachineLearningRating_v3.txt"

In [4]:
data = pd.read_csv(filepath_or_buffer=PATH_TO_DATA, delimiter='|', low_memory=False)

### Data Cleaning

1) Find missing values

In [6]:
# find columns with na values
column_na_rations = data.isna().mean()

# print out columns that have missing values and the amount of missing values as percentages
column_na_rations[column_na_rations > 0].sort_values(ascending=False) * 100

NumberOfVehiclesInFleet    100.000000
CrossBorder                 99.930207
CustomValueEstimate         77.956560
WrittenOff                  64.183810
Converted                   64.183810
Rebuilt                     64.183810
NewVehicle                  15.327998
Bank                        14.594670
AccountType                  4.022806
Gender                       0.953507
MaritalStatus                0.825819
mmcode                       0.055195
VehicleType                  0.055195
make                         0.055195
VehicleIntroDate             0.055195
NumberOfDoors                0.055195
bodytype                     0.055195
kilowatts                    0.055195
cubiccapacity                0.055195
Cylinders                    0.055195
Model                        0.055195
CapitalOutstanding           0.000200
dtype: float64

2) Handle missing values

- drop columns with a lot of missing values, i.e greater than 50%

In [12]:
# find the columns with missing values greater than 50%
to_be_droped = column_na_rations[column_na_rations > 0.5].keys()

# drop the columns
data = data.drop(columns=to_be_droped)

- removes data for column with missing values less than 5%

In [13]:
# find the columns with missing values less than 5%
to_be_droped = column_na_rations[column_na_rations < 0.05].keys()

# drop the columns
data = data.dropna(subset=to_be_droped)

- For columns in between use mode or mean(depending on column data type) to replace the missing values

In [21]:
# find the columns with missing values greater than 5% and less than 50%
to_be_replaced = column_na_rations[(column_na_rations >= 0.05) & (column_na_rations <=0.5)].keys()

Find the data type of those columns

In [22]:
data[[*to_be_replaced]].dtypes

Bank          object
NewVehicle    object
dtype: object

Both are categorical so I will use their respective modes

In [24]:
for column in to_be_replaced:
    # Get the mode of the column
    mode_value = data[column].mode()[0]  
    # Fill missing values with the mode
    data[column] = data[column].fillna(mode_value)  

Finally check for the missing values

In [25]:
# find columns with na values
column_na_rations = data.isna().mean()

# print out columns that have missing values and the amount of missing values as percentages
column_na_rations[column_na_rations > 0].sort_values(ascending=False) * 100

Series([], dtype: float64)

### Feature Engineering

1) I wanted to calculate the optimal premium values for every customer based on their TotalClaims. As this is a business money needs to be made and hence the premium price should at least give a return of 0, break even. So I will create a new feature , from TotalClaims and TotalPremium, called OptimumPremium.

In [33]:
# if the difference between totalPremium and totalClaims is negative then set the optimum premim = totalPremium + (TotalClaims - TotalPremium) else just make it equal to total premium
data['OptimalPremium'] = np.where((data['TotalPremium'] - data['TotalClaims'] < 0),
                                  (data['TotalPremium'] + (data['TotalClaims'] - data['TotalPremium'])),
                                  (data['TotalPremium'])
                                  )

Check the result of the top 10 rows

In [36]:
data[['TotalClaims', 'TotalPremium', 'OptimalPremium']].head(10)

,TotalClaims,TotalPremium,OptimalPremium
0,0.0,21.929825,21.929825
1,0.0,21.929825,21.929825
2,0.0,0.000000,0.000000
3,0.0,512.848070,512.848070
4,0.0,0.000000,0.000000
5,0.0,3.256435,3.256435
6,0.0,50.474737,50.474737
7,0.0,35.332316,35.332316
8,0.0,0.000000,0.000000
9,0.0,1.009474,1.009474
